In [1]:
import argparse
from datetime import datetime, timedelta, time
import random

import numpy as np

from fwa import FWA
from utils import load_data

SEED = 42
np.random.seed(SEED)
random.seed(SEED)

In [2]:
data = load_data(xml_path='data/ORTEC01.xml')
shifts = data['shifts']

def pad_time_string(tstr):
    parts = tstr.split(":")
    parts = [part.zfill(2) for part in parts]
    while len(parts) < 3:
        parts.append("00")
    return ":".join(parts)
shifts = dict(sorted(shifts.items(),
    key=lambda item: datetime.strptime(pad_time_string(item[1]["StartTime"]), "%H:%M:%S").time()
    if item[1]["StartTime"] else time.max))


shift_groups = data['shift_groups']
employees = data['employees']
contracts = data['contracts']
cover = data['cover_requirements']
off_reqs = data['shift_off_requests']
on_reqs = data['shift_on_requests']
start_date = data['start_date']  # datetime.date já
end_date = data['end_date']      # datetime.date já
cover_weights = data['cover_weights']

n_days = (end_date - start_date).days + 1
n_employees = len(employees)
shift_ids = list(shifts.keys())
n_shift_types = len(shift_ids)

In [8]:
cover

defaultdict(dict,
            {'Saturday': {'D': 2, 'E': 2, 'L': 2, 'N': 1},
             'Sunday': {'D': 2, 'E': 2, 'L': 2, 'N': 1},
             'Monday': {'D': 3, 'E': 3, 'L': 3, 'N': 1},
             'Tuesday': {'D': 3, 'E': 3, 'L': 3, 'N': 1},
             'Wednesday': {'D': 3, 'E': 3, 'L': 3, 'N': 1},
             'Thursday': {'D': 3, 'E': 3, 'L': 3, 'N': 1},
             'Friday': {'D': 3, 'E': 3, 'L': 3, 'N': 1}})

In [5]:
# Mapeamento para acesso rápido ao índice do funcionário
employee_id_to_index = {eid: i for i, eid in enumerate(employees.keys())}
# Mapeamento para acesso rápido ao índice do turno
shift_id_to_index = {sid: i for i, sid in enumerate(shift_ids)}

solution_size = n_employees * n_days
bounds = [(0 - 0.5, n_shift_types - 1 + 0.5)] * solution_size  # cada valor representa um turno possível

In [6]:
shift_id_to_index

{'E': 0, 'D': 1, 'L': 2, 'N': 3, 'OFF': 4}

In [7]:
shift_ids

['E', 'D', 'L', 'N', 'OFF']

In [ ]:
n_employees

In [ ]:
n_days

In [3]:
len(shifts)

5

In [4]:
for _, v in shifts.items():
    print(v)

{'Label': 'E', 'Name': 'Early', 'StartTime': '07:00:00', 'EndTime': '16:00:00', 'Color': 'Red'}
{'Label': 'D', 'Name': 'Day', 'StartTime': '08:00:00', 'EndTime': '17:00:00', 'Color': 'Lime'}
{'Label': 'L', 'Name': 'Late', 'StartTime': '14:00:00', 'EndTime': '23:00:00', 'Color': 'Blue'}
{'Label': 'N', 'Name': 'Night', 'StartTime': '23:00:00', 'EndTime': '07:00:00', 'Color': 'Gray'}
{'Label': 'OFF', 'Name': 'Folga', 'StartTime': None, 'EndTime': None}


In [ ]:
shifts_ordered = dict(
    sorted(
        shifts.items(),
        key=lambda item: datetime.strptime(item[1]["StartTime"], "%H:%M:%S") 
        if item[1]["StartTime"] else datetime.max
    )
)

In [ ]:
for _, v in shifts_ordered.items():
    print(v)

In [ ]:
shifts.keys()

In [ ]:
sorted(shifts.keys())

In [ ]:
def fitness(solution):
    return np.random.randint(0, 20000)

In [ ]:
fwa = FWA(func=fitness, 
              dim=solution_size, 
              bounds=bounds, 
              selection_method='distance',
              seed=SEED)

fwa.config(n=5, 
            m=50, 
            a = 0.04,
            b = 0.8,
            A_hat = 1.5,
            m_hat= 5,
            max_iter=100)

In [ ]:
fwa.init_fireworks()

In [ ]:
len(fwa.fireworks)

In [ ]:
fwa.fireworks[0].shape

In [ ]:
schedule = np.rint(fwa.fireworks[0]).astype(int).reshape((n_employees, n_days))

In [ ]:
schedule.shape

In [ ]:
for id, value in employees.items():
    print(id, value)

In [5]:
contracts['36']

[{'Max': {'Count': 20,
   'Weight': 1000.0,
   'WeightFunction': None,
   'Label': 'Max 20 shifts'},
  'Min': None,
  'RegionStart': None,
  'RegionEnd': None,
  'Pattern': [{'ShiftGroup': 'All'}]},
 {'Max': {'Count': 0,
   'Weight': 100.0,
   'WeightFunction': None,
   'Label': 'Min 2 consecutive free days'},
  'Min': None,
  'RegionStart': None,
  'RegionEnd': None,
  'Pattern': [{'Start': '0', 'Shift': '-', 'ShiftGroup': 'All'},
   {'ShiftGroup': 'All', 'Shift': '-'}]},
 {'Max': {'Count': 3,
   'Weight': 1000.0,
   'WeightFunction': None,
   'Label': 'Max 3 nights'},
  'Min': None,
  'RegionStart': None,
  'RegionEnd': None,
  'Pattern': [{'Shift': 'N'}]},
 {'Max': {'Count': 3,
   'Weight': 1000.0,
   'WeightFunction': None,
   'Label': 'Max 3 working weekends'},
  'Min': None,
  'RegionStart': None,
  'RegionEnd': None,
  'Pattern': [{'Start': '30', 'Shift': 'N'},
   {'StartDay': 'Saturday', 'ShiftGroup': 'All', 'Shift': '-'},
   {'StartDay': 'Saturday', 'Shift': '-', 'ShiftGroup':